In [1]:
import os
from datetime import datetime
from shutil import copyfile

In [2]:
def get_date(raw_file_name):
    return datetime.strptime(raw_file_name.split('covid19-')[1].replace('.csv', ''), '%Y-%m-%d')

In [3]:
def convert_back_to_str_file(date):
    return  'mexico-covid19-' + date.isoformat().split('T')[0] + '.csv'

In [4]:
available_files = [file for file in os.listdir() if 'mexico' in file] 
max_date = max([get_date(file) for file in available_files])

In [5]:
max_date_file = convert_back_to_str_file(max_date)

In [6]:
copyfile(max_date_file, 'latest.csv')

'latest.csv'